In [1]:
import json

In [2]:
def read_list_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    if isinstance(data, list):
        return data
    else:
        raise ValueError("The JSON file does not contain a list.")

In [16]:
data = read_list_from_json('hp/harry_potter_1_questions.json')

In [17]:
n_list, k_list, q_list = [], [], []
for d in data:
    if d['output'] is None:
        print(f"n = {d['history_upto']}, k = {d['context']}")
        continue
    n_list.append(d['history_upto'])
    k_list.append(d['context'])
    q_list.append(d['output'])
    print(f"number of questions: {len(d['output'])}")

number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questions: 10
number of questi